In [7]:
import os
os.chdir("/home/jennifer.badolato/watttime-python-client-aer-algo")

import math
import numpy as np
import pandas as pd
import datetime
import pytz
from datetime import datetime, timedelta
from watttime import WattTimeForecast, WattTimeHistorical

import optimizer.s3 as s3u
import evaluation.eval_framework as efu

import watttime.shared_anniez.alg.optCharger as optC
import watttime.shared_anniez.alg.moer as Moer

In [8]:
region = "PJM_NJ"
username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

actual_data = WattTimeHistorical(username, password)
hist_data = WattTimeForecast(username, password)

In [13]:
s3 = s3u.s3_utils()
key = '20240715_1k_synth_users.csv'
generated_data = s3.load_csvdataframe(file=key)

In [14]:
synth_data = generated_data.copy(deep=True)
synth_data = synth_data.head(1)

In [19]:
synth_data['moer_data'] = synth_data.apply(
    lambda x: efu.get_historical_fcst_data(
    x.plug_in_time,
    x.total_intervals_plugged_in,
    region = region
    ), axis = 1
)

In [30]:
synth_data['charger']= synth_data.apply(
    lambda x: efu.get_schedule_and_cost(
        x.MWh_fraction,
        x.charged_MWh_actual,
        math.ceil(x.total_intervals_plugged_in), # will throw an error if the plug in time is too shart to reach full charge, should soften to a warning
        x.moer_data,
        asap = True
        ), 
        axis = 1
        )

Greedy fit!
[0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.0003946666666666, 0.00039

In [36]:
synth_data['simple_fit_results'] = synth_data['charger'].apply(
    lambda  x: x.get_total_cost()
    )

In [37]:
synth_data[['projected_charging_cost_to_full','simple_fit_results']]

,projected_charging_cost_to_full,simple_fit_results
0,51.047331,51.047331
